In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import re
import string
import json
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

from sklearn.model_selection import train_test_split

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/test_brainstem/cleaned_data.csv")
df

,combined,สังคม,แรงงาน,ความมั่นคง,วัฒนธรรม,เศรษฐกิจ,สิทธิมนุษยชน,สิ่งแวดล้อม,การศึกษา,คุณภาพชีวิต,ต่างประเทศ,ไอซีที,การเมือง
0,อรรถจักร์ สัตยานุรักษ์ จินตนาการภาษา ภาษา สื่...,0,0,0,1,0,0,0,1,0,0,0,1
1,กสทชเร่งทรู แจ้งสิทธิผู้ใช้บริการยกเลิก HBO 6 ...,0,0,0,0,0,0,0,0,1,0,1,0
2,ศาลฎีกาประกัน เฉลียว ช่างตัดเสื้อคดี 112 นอนคุ...,0,0,0,0,0,1,0,0,0,0,0,1
3,มูลนิธิผู้บริโภคชวนประกวดสปอตโทรทัศน์ชิงเงินแส...,0,0,0,0,0,0,0,0,1,0,0,0
4,กสทชร้อง ปลัดคลัง พิจารณาการกระทำรองปลัด ประมู...,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,‘UnionBook’ เครือข่ายสังคมออนไลน์ สำหรับ “สหภา...,1,1,0,0,0,0,0,0,1,0,0,0
4496,คณะรัฐมนตรีเห็นชอบร่าง พรบหอพักแยกหอพักชายหญิง...,0,0,1,0,1,0,0,1,1,0,0,0
4497,นปช แถลงการณ์ห่วงใย พันธมิตร แนวร่วมประชาธิปไต...,0,0,0,0,0,0,0,0,0,0,0,1
4498,ศาลปกครองรับฟ้อง ลักษณ์ ฟ้อง ประยุทธ์ กรณีชดใช...,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 2e-5
# tokenizer = AutoTokenizer.from_pretrained('roberta-base')
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [ ]:
target_cols = [col for col in df.columns if col not in ['combined']]
target_cols

['สังคม',
 'แรงงาน',
 'ความมั่นคง',
 'วัฒนธรรม',
 'เศรษฐกิจ',
 'สิทธิมนุษยชน',
 'สิ่งแวดล้อม',
 'การศึกษา',
 'คุณภาพชีวิต',
 'ต่างประเทศ',
 'ไอซีที',
 'การเมือง']

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.combined
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Set the seed for reproducibility
random_state = 42

# Shuffle the DataFrame and reset the index
shuffled_df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)

# Define the proportion of data to use for training
train_ratio = 0.8

# Calculate the number of rows for training and testing
train_size = int(train_ratio * len(shuffled_df))
test_size = len(shuffled_df) - train_size

# Split the DataFrame into train and test sets
train_df = shuffled_df[:train_size]
test_df = shuffled_df[train_size:]

# Reset the index of the train and test DataFrames
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
train_df.shape

(3600, 13)

In [ ]:
test_df.shape

(900, 13)

In [ ]:
train_dataset = BERTDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = BERTDataset(test_df, tokenizer, MAX_LEN)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE,
                          num_workers=4, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE,
                          num_workers=4, shuffle=False, pin_memory=True)

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert = AutoModel.from_pretrained('bert-base-multilingual-uncased')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,12)
        # 768 is size for this pretrain
        # 12 is number of the classes

    def forward(self, ids, mask, token_type_ids):
        _, features = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
#         output_2 = self.l2(output_1)
        output = self.fc(features)
        return output

model = BERTClass()
model.to(device);

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(train_loader, 0):
        #print(1)
        ids = data['ids'].to(device, dtype = torch.long)
        #print("ids", ids.shape)
        mask = data['mask'].to(device, dtype = torch.long)
        #print("mask", mask.shape)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        #print("tk", token_type_ids.shape)
        targets = data['targets'].to(device, dtype = torch.float)
        #print(2)

        outputs = model(ids, mask, token_type_ids)
        #print(3)
        #print(outputs.shape)

        loss = loss_fn(outputs, targets)
        #print(4)
        if _%500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        #print(5)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/test_brainstem/bert_model.bin')